In [41]:
from multiprocessing import Process, Manager
import multiprocessing as mp
from librosa import stft
import numpy as np
from time import time

In [42]:
waveforms = np.load('y_list.npy')

# Multi Processing for librosa

In [50]:
n_workers = 40

In [51]:
def foo(i):  # the managed list `L` passed explicitly.
    spec = stft(i)
    return spec

## With appending function (free order)

In [52]:
start = time()
n_workers = mp.cpu_count()
pool = mp.Pool(n_workers)
result = []
for filename in waveforms:
    pool.apply_async(foo, args=(filename, ), callback=result.append)
pool.close()  # no more jobs will be added
pool.join()
print('{:2f} seconds are used to finish the conversion'.format(time()-start))

11.595662 seconds are used to finish the conversion


## With appending function (ensure in order)

In [40]:
start = time()
n_workers = mp.cpu_count()
pool = mp.Pool(n_workers)
result = []
for filename in waveforms:
    r = pool.apply_async(foo, args=(filename, ), callback=result.append)
    r.wait()
pool.close()  # no more jobs will be added
pool.join()
print('{:2f} seconds are used to finish the conversion'.format(time()-start))

28.850948 seconds are used to finish the conversion


# Multi Processing approach

In [4]:
start = time()
manager = Manager()
L = manager.list()  # <-- can be shared between processes.
processes = []
for i in waveforms:
    p = Process(target=dothing, args=(L,i))  # Passing the list
    p.start()
    processes.append(p)
for p in processes:
    p.join()
#     print(L)
print('{:2f} seconds are used to finish the conversion'.format(time()-start))

42.939029 seconds are used to finish the conversion
42.939029 seconds are used to finish the conversion
42.939029 seconds are used to finish the conversion


# Multi Processing Pool approach

In [5]:
from multiprocessing import Pool

def f(x):
    return stft(x)

start = time()
p = Pool(n_workers)
p.map(f, waveforms)
print('{:2f} seconds are used to finish the conversion'.format(time()-start))

13.009506 seconds are used to finish the conversion
13.009506 seconds are used to finish the conversion
13.009506 seconds are used to finish the conversion


# For loop approach

In [46]:
L2 = []
start = time()
for i in waveforms:
    L2.append(stft(i))
print('{:2f} seconds are used to finish the conversion'.format(time()-start))    

9.738493 seconds are used to finish the conversion


# Comparing the results

In [37]:
counter = 0
for i in range(len(result)):
    if np.array_equal(result[i], L2[i]) == True:
        counter += 1

In [38]:
counter

1770